In [1]:
import ipywidgets as widgets

In [2]:
#Przykładowy wybór botów
bot1_Select = widgets.Select(
    options=['random', 'alfabeta', 'MCTS', 'inny MCTS'],
    value='random',
    description='bot 1:',
    disabled=False
)

bot2_Select = widgets.Select(
    options=['random', 'alfabeta', 'MCTS', 'inny MCTS'],
    value='random',
    description='bot 2:',
    disabled=False
)

In [3]:
#Przykładowy wybór stanu początkowego
stacks_values_widget = widgets.Text(
    value="",
    description='Wartości kolejnych stosów:',
    placeholder='Np. 10 10 10',
    style = {'description_width': 'initial'})

In [4]:
#Przykładowy wybór czasu na ruch
time_for_move_widget = widgets.FloatText(
    value=2,
    description='Czas na ruch w sekundach:',
    style = {'description_width': 'initial'})

In [11]:
#Alpha beta algorytm
import math
import time

def alphabeta_move(stacks_values, time_for_move):
    start_time = time.time()
    for i in range(1, sum(stacks_values) + 1):
        stack, val = alphabeta_move_timed(stacks_values, i, start_time + time_for_move)
        if(val > 0):
            chosen_stack = stack
            chosen_val = val
        else:
            break
    return chosen_stack, chosen_val
    
def alphabeta_move_timed(stacks_values, depth, end_time):
    max_temp = -math.inf
    for stack in range(len(stacks_values)):
        if(stacks_values[stack] > 0):
            chosen_stack = stack
            chosen_val = 1
            break
    for stack in range(len(stacks_values)):
        for i in range(1, stacks_values[stack]+1):  
            stacks_values[stack] =  stacks_values[stack] - i
            temp = alphabeta_move2(stacks_values, depth, -math.inf, math.inf, -1, end_time)
            stacks_values[stack] =  stacks_values[stack] + i
            if(max_temp <= temp):
                max_temp = temp
                chosen_stack = stack
                chosen_val = i
            if(time.time() > end_time):
                return 0, 0 
    return chosen_stack, chosen_val
    

def alphabeta_move2(stacks_values, depth, alfa, beta, player, end_time):
    if(time.time() > end_time):
        return 0
    n = len(stacks_values)
    if stacks_values.count(0) == n:
        if(player == 1):
            return 1
        else:
            return -1
    if(depth == 0):
        return 0
    if player == -1:
        for stack in range(n):
            for i in range(1, stacks_values[stack]+1):  
                stacks_values[stack] =  stacks_values[stack] - i
                beta = min(beta, alphabeta_move2(stacks_values, depth-1, alfa, beta, 1, end_time))
                stacks_values[stack] =  stacks_values[stack] + i
                if alfa >= beta or alfa == 1:
                    break
            if alfa >= beta or alfa == 1:
                    break
        return beta
    else:
        for stack in range(n):
            for i in range(1, stacks_values[stack]+1):  
                stacks_values[stack] =  stacks_values[stack] - i
                alfa = max(alfa, alphabeta_move2(stacks_values, depth-1, alfa, beta, -1, end_time))
                stacks_values[stack] =  stacks_values[stack] + i
                if alfa >= beta or beta == -1:
                    break
            if alfa >= beta or beta == -1:
                    break
        return alfa
    
            


In [10]:
import random

def random_move(stacks_values, time_for_move):
    n = len(stacks_values)
    while(True):
        chosen_stack = random.randint(0, n-1)
        if(stacks_values[chosen_stack] > 0):
            val = random.randint(1, stacks_values[chosen_stack])
            return chosen_stack,  val

# Przykładowy przebieg rozgrywki
def simulate_game():
    stacks_values = list(map(int, stacks_values_widget.value.split()))
    n = len(stacks_values)
    empty_stack_counter = 0
    player = 1
    
    if(bot1_Select.value == 'random'):
        bot1_move = random_move
    elif (bot1_Select.value == 'alfabeta'):
        bot1_move = alphabeta_move
    
    if(bot2_Select.value == 'random'):
        bot2_move = random_move
    if(bot2_Select.value == 'alfabeta'):
        bot2_move = alphabeta_move
    

    while(True):
        print(stacks_values)
        if(player == 1):
            chosen_stack,  val =  bot1_move(stacks_values, time_for_move_widget.value)
        else:
            chosen_stack,  val =  bot2_move(stacks_values, time_for_move_widget.value)
        
        print("Player", player, "take", val, "from stack", chosen_stack)
        stacks_values[chosen_stack] = stacks_values[chosen_stack] - val
        if(stacks_values[chosen_stack] == 0):
            empty_stack_counter =  empty_stack_counter+1
        player = 1 if player == 2 else 2
        if(empty_stack_counter == n):
            print("Player", player, "wins")
            break

    

In [7]:
from IPython.display import clear_output

#Przycisk startu

button = widgets.Button(
    description='Start',
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        simulate_game()

button.on_click(on_button_clicked)

In [8]:
display(bot1_Select)
display(bot2_Select)

display(stacks_values_widget)
display(time_for_move_widget)

display(button, output)

Select(description='bot 1:', options=('random', 'alfabeta', 'MCTS', 'inny MCTS'), value='random')

Select(description='bot 2:', options=('random', 'alfabeta', 'MCTS', 'inny MCTS'), value='random')

Text(value='', description='Wartości kolejnych stosów:', placeholder='Np. 10 10 10', style=DescriptionStyle(de…

FloatText(value=2.0, description='Czas na ruch w sekundach:', style=DescriptionStyle(description_width='initia…

Button(description='Start', style=ButtonStyle())

Output()